### Main notebook

In [1]:
import pandas as pd
import numpy as np
import json
import networkx as nx
from tqdm import tqdm
from compound import Compound
from reaction import Reaction
from graph import Graph
from data import Data

# suppres rdkit warnings
import rdkit
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

# read data from csv
cpds = pd.read_csv('../GNN_toxic/data/raw/compounds_final.csv', index_col=0) # containing toxicity
rxns = pd.read_csv('data/reactions_final.csv', index_col=0)
pairs = pd.read_csv('data/pairs_final.csv', index_col=0)
cofactors = pd.read_csv('data/original/cofactors_KEGG.csv')

# create class instances
data = Data()
graph = Graph(pairs=pairs)

# Create a Compound object for each row in the DataFrame and add it to the data
for index, row in cpds.iterrows():
    entry = row['Entry']
    name = row['Names']
    formula = row['Formula']
    mw = row['mol_weight']
    smiles = row['SMILES']
    is_cofactor = row['Entry'] in cofactors['Entry'].values
    is_toxic = row['toxic']

    compound = Compound(entry, name, formula, mw, smiles, is_cofactor, is_toxic)
    data.add_element('compound', compound)

# Create a Reaction object for each row in the DataFrame and add it to the data
for index, row in rxns.iterrows():
    entry = row['Entry']
    name = row['Names']
    compounds = row['Compound']
    enzyme = row['EC Number']

    reaction = Reaction(entry, name, compounds, enzyme)
    data.add_element('reaction', reaction)

In [2]:
# CREATE GRAPH
graph.create_graph(data=data, pairs=pairs)

graph.calculate_edge_mol_weight(data)
graph.calculate_smiles_similarity(data)

# nodes: 8591 
# edges: 30026


100%|██████████| 30026/30026 [00:42<00:00, 706.27it/s] 


In [20]:
graph.G.edges()[('C00013', 'C00009')]

{'mol_weight': 10, 'smiles_similarity': 0.74}

In [26]:
s = 'C00082'
t = 'C00811'

pairs[(pairs['source']==s) & (pairs['target']==t)].iloc[0]['Reaction']

'R00737'

In [ ]:
######### VALIDATION SET FROM nicepath ###########
test_cases = pd.read_csv('data/original/test_cases.csv')
test_cases['source'] = test_cases['Pathway '].apply(lambda x: x.split(',')[0])
test_cases['target'] = test_cases['Pathway '].apply(lambda x: x.split(',')[len(x.split(','))-1])
test_cases['paths_list'] = test_cases['Pathway '].apply(lambda x: x.split(','))

paths = graph.validate(test_cases, 'mol_weight')

In [ ]:
paths

In [ ]:
''' Get a specific edge from networkx graph '''


In [ ]:
graph.G.edges()

### display pathway to BIGG style

In [ ]:
for pathway in paths['Pathway']:
    for cpd in pathway:
        print(cpd)
        graph.kegg_to_bigg_compound(cpd)        
    break

In [ ]:
print(test_cases.iloc[20]['Pathway '])
print(test_cases.iloc[42]['Pathway '])
print(test_cases.iloc[47]['Pathway '])

In [ ]:
kegg_cpd = 'C00084'
bigg_cpd = graph.kegg_to_bigg_compound(kegg_cpd)
print(bigg_cpd)

kegg_rxn = 'R00084'
bigg_rxn = graph.kegg_to_bigg_compound(kegg_rxn)
print(bigg_rxn)

In [ ]:
1/0

In [ ]:
p, smiles_smi, idx, comm_changes, idxcom = graph.constrained_shortest_path('C00082', 'C01533', weight='mol_weight')
for i in p: print(i)

In [ ]:
comm_changes

In [ ]:
smiles_smi

### Study graph

### Try cluster graph nodes

### Try similarity based on SMILES

In [ ]:
smiles1 = data.get_compound_by_id('C00223').smiles
smiles2 = data.get_compound_by_id('C00323').smiles

from rdkit import Chem
from rdkit import DataStructs

ms = [Chem.MolFromSmiles(smiles1), Chem.MolFromSmiles(smiles2)]
fs = [Chem.RDKFingerprint(x) for x in ms]
s = DataStructs.FingerprintSimilarity(fs[0], fs[1])
print('Similarity: '+str(s))


smiles1 = data.get_compound_by_id('C00223').smiles
smiles2 = data.get_compound_by_id('C12096').smiles

from rdkit import Chem
from rdkit import DataStructs

ms = [Chem.MolFromSmiles(smiles1), Chem.MolFromSmiles(smiles2)]
fs = [Chem.RDKFingerprint(x) for x in ms]
s = DataStructs.FingerprintSimilarity(fs[0], fs[1])
print('Similarity: '+str(s))